<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import torch
from torch.utils.data.dataset import random_split
from torchaudio import datasets

from dcunet.dcunet import DCUnet10
from dcunet.train import train, wsdr_fn, wsdr_on_processed_data, wsdr_on_noisy_data
from utils.data import get_data, preprocess, WrappedDataLoader, StreetNoiseDataLoader

In [2]:
import matplotlib.pyplot as plt
import IPython.display as ipd

In [3]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

dataset = datasets.VCTK_092(root="data", download=False)

ds_size = len(dataset)
train_i = int(0.1 * ds_size)
train_i += train_i % 2
val_i = int(0.11 * ds_size) - train_i
val_i += val_i % 2
test_i = int(0.12 * ds_size) - train_i - val_i
test_i += test_i % 2
other = ds_size - train_i - val_i - test_i

train_ds, val_ds, test_ds, other = random_split(dataset, lengths=[train_i, val_i, test_i, other],
                                                generator=torch.Generator().manual_seed(42))

batch_size = 1
DEVICE = dev
SAMPLE_RATE = 48000
N_FFT = SAMPLE_RATE * 64 // 1000 + 4
HOP_LENGTH = SAMPLE_RATE * 16 // 1000 + 4

noise_files = ["noise/1.wav", "noise/2.wav", "noise/3.wav"]

train_dl, val_dl, test_dl = get_data(train_ds, val_ds, test_ds, batch_size)
train_dl = StreetNoiseDataLoader(train_dl, preprocess, HOP_LENGTH, dev, noise_files, 0.5)
val_dl = StreetNoiseDataLoader(val_dl, preprocess, HOP_LENGTH, dev, noise_files, 0.5)
test_dl = StreetNoiseDataLoader(test_dl, preprocess, HOP_LENGTH, dev, noise_files, 0.5)

dcunet10 = DCUnet10(N_FFT, HOP_LENGTH).to(dev)

loss_fn = wsdr_fn
optimizer = torch.optim.Adam(dcunet10.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [ ]:
train_losses, test_losses = train(dcunet10, train_dl, val_dl, loss_fn, optimizer, scheduler, 100, dev)

In [10]:
xi = iter(val_dl)

In [29]:
x = next(xi)
x_noisy = x[0][0]
x_true = x[1][0]
with torch.no_grad():
    x_pred = net(x[0][:1, :])
x_pred = x_pred / (x_pred.max() - x_pred.min()) * (x_true.max() - x_true.min())

In [30]:
display(ipd.Audio(x_noisy.squeeze().cpu(), rate=SAMPLE_RATE));

In [31]:
display(ipd.Audio(x_true.squeeze().cpu(), rate=SAMPLE_RATE));

In [32]:
display(ipd.Audio(x_pred.squeeze().cpu(), rate=SAMPLE_RATE));

In [6]:
torch.save(dcunet10.state_dict(), "dcu_street.pth")

In [9]:
net = DCUnet10(N_FFT, HOP_LENGTH).to(dev)
net.load_state_dict(torch.load("dcu_street.pth"))

<All keys matched successfully>

wSDR на отложенной выборке после зашумления

In [15]:
wsdr_on_noisy_data(test_dl, dev)

-0.3853401779345998

wSDR на отложенной выборке после обработки сетью

In [16]:
wsdr_on_processed_data(test_dl, dev, net)

-0.9385966974306325